## Unit 2 - Part 3a: Chain of Thought (CoT)

# 2. Concept: Latent Reasoning

In [1]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 9.5 MB/s eta 0:00:00
Enter your Groq API Key: ··········


## 3. The Experiment: A Tricky Math Problem

**Problem:**
"Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

In [2]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [3]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


## 4. Analysis

Look at the output. By explicitly breaking it down:
1.  "Roger starts with 5."
2.  "2 cans * 3 balls = 6 balls."
3.  "5 + 6 = 11."

The model effectively "debugs" its own logic by generating the intermediate steps.

## Unit 2 - Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

## 2. Tree of Thoughts (ToT)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a Child Psychologist, I would recommend **Solution 2: The "Veggie Treasure Hunt"** as the most sustainable approach to encourage a 5-year-old to eat vegetables. Here's why:

1. **Gradual exposure**: This approach allows for gradual exposure to vegetables, which is essential for children's developing tastes and digestive systems. It reduces the likelihood of overwhelming or rejecting new foods.
2. **Involvement and agency**: By involving your child in meal prep and letting them help discover the hidden veggies, you're giving them a sense of agency and control over their food choices. This can foster a positive relationship with food and vegetables.
3. **Subtle introduction**: Hiding finely chopped or shredded vegetables in familiar foods makes them less noticeable, reducing the likelihood of your child resisting or protesting.
4. **Building trust**: As your child becomes accustomed to eating hidden veggies, they'll begin to trust that they're get

## 3. Graph of Thoughts (GoT)

In [6]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
"In 'Chronocide,' a brilliant physicist, Emma Taylor, discovers a way to manipulate time, but her invention unleashes a malevolent entity from the past, a vengeful spirit of a 19th-century serial killer, who was thought to have been executed but was actually sent back in time by an earlier experiment. As Emma navigates a complex web of past and future events to prevent a catastrophic future, she finds herself falling for a charming historical figure, James, who was the killer's intended victim. But as their love grows stronger, they realize that James' past is inextricably linked to the entity's return, and that their love may be the key to preventing a chronocide - a catastrophic event that will erase entire eras from existence, leaving only a desolate, dark future in its wake."


## 4. Summary & Comparison Table

| Method | Structure | Best For... | Cost/Latency |
|--------|-----------|-------------|--------------|
| **Simple Prompt** | Input -> Output | Simple facts, summaries | ⭐ Low |
| **CoT (Chain)** | Input -> Steps -> Output | Math, Logic, Debugging | ⭐⭐ Med |
| **ToT (Tree)** | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming | ⭐⭐⭐ High |
| **GoT (Graph)** | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ V. High |

**Recommendation:** Start with CoT. Only use ToT/GoT if CoT fails.